In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input

import tfcoreml

In [2]:
keras_model = Sequential([
    Dense(1, input_shape=[1]),
])

keras_model.save('./model.h5')

In [3]:
print(keras_model.input_shape)

# get input, output node names for the TF graph from the Keras model
input_name = keras_model.inputs[0].name.split(':')[0]
keras_output_node_name = keras_model.outputs[0].name.split(':')[0]
graph_output_node_name = keras_output_node_name.split('/')[-1]

print(input_name)
print(keras_output_node_name)
print(graph_output_node_name)

core_model = tfcoreml.convert(tf_model_path='./model.h5',
                         input_name_shape_dict={input_name: (1, 1)},
                         output_feature_names=[graph_output_node_name],
                         minimum_ios_deployment_target='13')
core_model.save('./model.mlmodel')

core_model

(None, 1)
dense_input
dense/Identity
Identity


0 assert nodes deleted
['sequential/dense/MatMul/ReadVariableOp:0', 'sequential/dense/BiasAdd/ReadVariableOp/resource:0', 'sequential/dense/BiasAdd/ReadVariableOp:0', 'sequential/dense/MatMul/ReadVariableOp/resource:0']
2 nodes deleted
0 nodes deleted
0 nodes deleted
[Op Fusion] fuse_bias_add() deleted 2 nodes.
2 identity nodes deleted
1 disconnected nodes deleted
[SSAConverter] Converting function main ...
[SSAConverter] [1/2] Converting op type: 'Placeholder', name: 'dense_input', output_shape: (1, 1).
[SSAConverter] [2/2] Converting op type: 'MatMul', name: 'Identity', output_shape: (1, 1).


input {
  name: "dense_input"
  type {
    multiArrayType {
      shape: 1
      shape: 1
      dataType: DOUBLE
    }
  }
}
output {
  name: "Identity"
  type {
    multiArrayType {
      dataType: DOUBLE
    }
  }
}
metadata {
  userDefined {
    key: "coremltoolsVersion"
    value: "3.1"
  }
}

In [4]:
spec = core_model.get_spec()
spec

specificationVersion: 4
description {
  input {
    name: "dense_input"
    type {
      multiArrayType {
        shape: 1
        shape: 1
        dataType: DOUBLE
      }
    }
  }
  output {
    name: "Identity"
    type {
      multiArrayType {
        dataType: DOUBLE
      }
    }
  }
  metadata {
    userDefined {
      key: "coremltoolsVersion"
      value: "3.1"
    }
  }
}
neuralNetwork {
  layers {
    name: "Identity"
    input: "dense_input"
    output: "Identity"
    inputTensor {
      rank: 2
      dimValue: 1
      dimValue: 1
    }
    outputTensor {
      rank: 2
      dimValue: 1
      dimValue: 1
    }
    batchedMatmul {
      weightMatrixFirstDimension: 1
      weightMatrixSecondDimension: 1
      hasBias: true
      weights {
        floatValue: -0.4110192060470581
      }
      bias {
        floatValue: 0.0
      }
    }
  }
  arrayInputShapeMapping: EXACT_ARRAY_MAPPING
  imageInputShapeMapping: RANK4_IMAGE_MAPPING
}

In [ ]:
data = {'dense_input': [1.0, 1.0, 1.0]}
predictions = core_model.predict(data)